In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import time
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Phylo import draw
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Align import MultipleSeqAlignment
import numpy as np

# Functions for Algorithms

def extract_sequences_from_pdb_file(file_path):
    """Extract sequences from PDB files."""
    sequences = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("SEQRES"):
                sequence = line.split()[4:]
                sequences.extend(sequence)
    return ''.join(sequences)

def needleman_wunsch(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2)
    return alignments[0]

def smith_waterman(seq1, seq2):
    alignments = pairwise2.align.localxx(seq1, seq2)
    return alignments[0]

def dali_alignment(seq1, seq2):
    distance_matrix = np.zeros((len(seq1), len(seq2)))
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            distance_matrix[i, j] = 0 if seq1[i] == seq2[j] else 1
    alignment_score = np.sum(distance_matrix)
    return alignment_score, distance_matrix

def burrows_wheeler_transform(text):
    rotations = [text[i:] + text[:i] for i in range(len(text))]
    rotations.sort()
    return ''.join(rot[-1] for rot in rotations)

def levenshtein_distance(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = np.zeros((m + 1, n + 1), dtype=int)
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if seq1[i - 1] == seq2[j - 1] else 1
            dp[i, j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)

    return dp[m][n]

def wagner_fischer(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

def kmp_match(text, pattern):
    m, n = len(pattern), len(text)
    pi = [0] * m
    j = 0

    for i in range(1, m):
        while j > 0 and pattern[i] != pattern[j]:
            j = pi[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        pi[i] = j

    matches = []
    j = 0
    for i in range(n):
        while j > 0 and text[i] != pattern[j]:
            j = pi[j - 1]
        if text[i] == pattern[j]:
            j += 1
        if j == m:
            matches.append(i - m + 1)
            j = pi[j - 1]
    return matches

def phylogenetic_tree(sequences):
    alignment = MultipleSeqAlignment([SeqRecord(Seq(seq), id=f"Seq{i+1}") for i, seq in enumerate(sequences)])
    calculator = DistanceCalculator('identity')
    dm = calculator.get_distance(alignment)
    constructor = DistanceTreeConstructor()
    tree = constructor.nj(dm)
    return tree

def ukkonen_edit_distance(seq1, seq2):
    m, n = len(seq1), len(seq2)
    prev_row = list(range(n + 1))
    curr_row = [0] * (n + 1)
    for i in range(1, m + 1):
        curr_row[0] = i
        for j in range(1, n + 1):
            cost = 0 if seq1[i - 1] == seq2[j - 1] else 1
            curr_row[j] = min(prev_row[j] + 1, curr_row[j - 1] + 1, prev_row[j - 1] + cost)
        prev_row = curr_row[:]
    return curr_row[n]

def dynamic_time_warping(seq1, seq2):
    m, n = len(seq1), len(seq2)
    dp = np.zeros((m + 1, n + 1), dtype=float)
    dp[0, 1:] = np.inf
    dp[1:, 0] = np.inf

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = abs(ord(seq1[i - 1]) - ord(seq2[j - 1]))
            dp[i, j] = cost + min(dp[i - 1, j], dp[i, j - 1], dp[i - 1, j - 1])

    return dp[m, n]

# GUI Functions

def run_alignment():
    file1 = entry_file1.get()
    file2 = entry_file2.get()

    try:
        seq1 = extract_sequences_from_pdb_file(file1)
        seq2 = extract_sequences_from_pdb_file(file2)

        algorithm = selected_algorithm.get()
        start_time = time.time()

        if algorithm == "Needleman-Wunsch":
            alignment = needleman_wunsch(seq1, seq2)
            align_score = alignment[2]
            result = format_alignment(*alignment)
        elif algorithm == "Smith-Waterman":
            alignment = smith_waterman(seq1, seq2)
            align_score = alignment[2]
            result = format_alignment(*alignment)
        elif algorithm == "DALI":
            align_score, _ = dali_alignment(seq1, seq2)
            result = f"DALI Alignment Score: {align_score}"
        elif algorithm == "Burrows-Wheeler Transform (BWT)":
            result = burrows_wheeler_transform(seq1)
            align_score = len(result)
            result = f"BWT Result: {result[:100]}..."  # Show only first 100 characters for brevity
        elif algorithm == "Levenshtein Distance":
            align_score = levenshtein_distance(seq1, seq2)
            result = f"Levenshtein Distance: {align_score}"
        elif algorithm == "Wagner-Fischer":
            align_score = wagner_fischer(seq1, seq2)
            result = f"Wagner-Fischer Distance: {align_score}"
        elif algorithm == "Knuth-Morris-Pratt (KMP)":
            matches = kmp_match(seq1, seq2)
            align_score = len(matches)
            result = f"KMP Pattern Matches at positions: {matches}"
        elif algorithm == "Ukkonen’s Algorithm":
            align_score = ukkonen_edit_distance(seq1, seq2)
            result = f"Ukkonen’s Distance: {align_score}"
        elif algorithm == "Dynamic Time Warping (DTW)":
            align_score = dynamic_time_warping(seq1, seq2)
            result = f"DTW Distance: {align_score}"
        elif algorithm == "Phylogenetic Tree Construction":
            tree = phylogenetic_tree([seq1, seq2])
            result = f"Phylogenetic Tree:\n{tree}"
        else:
            raise ValueError("Invalid algorithm selected.")

        elapsed_time = time.time() - start_time
        result_text.delete("1.0", tk.END)
        result_text.insert(tk.END, f"Algorithm: {algorithm}\n")
        result_text.insert(tk.END, f"Result: {result}\n")
        result_text.insert(tk.END, f"Time Taken: {elapsed_time:.4f} seconds\n")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

def browse_file1():
    filename = filedialog.askopenfilename(title="Select Sequence 1 File", filetypes=(("PDB files", "*.pdb"), ("All files", "*.*")))
    if filename:
        entry_file1.delete(0, tk.END)
        entry_file1.insert(0, filename)

def browse_file2():
    filename = filedialog.askopenfilename(title="Select Sequence 2 File", filetypes=(("PDB files", "*.pdb"), ("All files", "*.*")))
    if filename:
        entry_file2.delete(0, tk.END)
        entry_file2.insert(0, filename)

# Main GUI Setup
root = tk.Tk()
root.title("Sequence Alignment and Comparison Tool")

# Algorithm Selection
selected_algorithm = tk.StringVar(value="Needleman-Wunsch")
algorithms = [
    "Needleman-Wunsch", "Smith-Waterman", "DALI", "Burrows-Wheeler Transform (BWT)",
    "Levenshtein Distance", "Wagner-Fischer", "Knuth-Morris-Pratt (KMP)",
    "Ukkonen’s Algorithm", "Dynamic Time Warping (DTW)", "Phylogenetic Tree Construction"
]

algorithm_menu = tk.OptionMenu(root, selected_algorithm, *algorithms)
algorithm_menu.grid(row=0, column=1)

# File Input
tk.Label(root, text="Sequence 1 (PDB):").grid(row=1, column=0)
entry_file1 = tk.Entry(root, width=50)
entry_file1.grid(row=1, column=1)
browse_button1 = tk.Button(root, text="Browse", command=browse_file1)
browse_button1.grid(row=1, column=2)

tk.Label(root, text="Sequence 2 (PDB):").grid(row=2, column=0)
entry_file2 = tk.Entry(root, width=50)
entry_file2.grid(row=2, column=1)
browse_button2 = tk.Button(root, text="Browse", command=browse_file2)
browse_button2.grid(row=2, column=2)

# Run Button
run_button = tk.Button(root, text="Run Alignment", command=run_alignment)
run_button.grid(row=3, column=1)

# Results Display
result_text = tk.Text(root, height=10, width=70)
result_text.grid(row=4, column=0, columnspan=3)

root.mainloop()
